<a href="https://colab.research.google.com/github/RigvedRocks/Face-Recognition-System/blob/main/Facial_Recognition_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Downloading MTCNN for Face Detection**

In [ ]:
!pip install mtcnn
import mtcnn
print(mtcnn.__version__)

In [ ]:
import os
from os import listdir
from os.path import isdir
from PIL import Image
import matplotlib.pyplot as plt
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import warnings
warnings.filterwarnings('ignore')

# **Face Detection**

In [ ]:
def extract_face(filename , required_size = (160,160)):
       image = Image.open(filename)
       image = image.convert('RGB')
       pixels = asarray(image)
       detector = MTCNN()
       results = detector.detect_faces(pixels)
       x1,y1,width,height = results[0]['box']
       x1,y1 = abs(x1),abs(y1)
       x2,y2 = x1 + width,y1 + height
       face = pixels[y1:y2,x1:x2]
       image = Image.fromarray(face)
       image = image.resize(required_size)
       face_array = asarray(image)
       return face_array

In [ ]:
def load_faces(directory):
      faces = list()
      for filename in listdir(directory):
         path = directory + filename
         face = extract_face(path)
         faces.append(face)
      return faces

In [ ]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)

In [ ]:
trainX, trainy = load_dataset('/content/drive/My Drive/Colab/Five_Faces/train/')
print(trainX.shape, trainy.shape)

In [ ]:
testX, testy = load_dataset('/content/drive/My Drive/Colab/Five_Faces/val/')
print(testX.shape , testy.shape)

In [ ]:
savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

# **Calculating Face Embeddings**

In [ ]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model


In [ ]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
# load the face dataset
data = load('5-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
# load the facenet model
model = load_model('/content/drive/My Drive/Colab/facenet_keras/facenet_keras.h5')
print('Loaded Model')

In [ ]:
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

In [ ]:
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

In [ ]:
# save arrays to one file in compressed format
savez_compressed('5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

# **Creating Face Classifier**

In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load dataset
data = load('5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

In [ ]:
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

In [ ]:
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

In [ ]:
# fit model
svc = SVC(kernel='linear', probability=True)
svc.fit(trainX, trainy)

In [ ]:
# predict
yhat_train = svc.predict(trainX)
yhat_test = svc.predict(testX)

In [ ]:
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)

In [ ]:
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
# load faces
data = load('5-celebrity-faces-dataset.npz')
testX_faces = data['arr_2']

# **Testing model on an random example**

In [ ]:
# test model on a random example from the test dataset
from random import choice
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

In [ ]:
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = svc.predict(samples)
yhat_prob = svc.predict_proba(samples)

In [ ]:
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)

In [ ]:
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])

In [ ]:
# plot for fun
plt.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
plt.title(title)
plt.show()